# Защита персональных данных клиентов

Вам нужно защитить данные клиентов страховой компании «Хоть потоп». Разработайте такой метод преобразования данных, чтобы по ним было сложно восстановить персональную информацию. Обоснуйте корректность его работы.

Нужно защитить данные, чтобы при преобразовании качество моделей машинного обучения не ухудшилось. Подбирать наилучшую модель не требуется.

## Загрузка данных

Импортируем необходимые модули.

In [1]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score

Введем константы.

In [2]:
SEED = 1505

Загрузим данные.

In [3]:
df = pd.read_csv("datasets/insurance.csv")
display(df.head())
print("Размер датасета:", df.shape)
display(df.info())
df.describe()

,Пол,Возраст,Зарплата,Члены семьи,Страховые выплаты
0,1,41.0,49600.0,1,0
1,0,46.0,38000.0,1,1
2,0,29.0,21000.0,0,0
3,0,21.0,41700.0,2,0
4,1,28.0,26100.0,0,0


Размер датасета: (5000, 5)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 5 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Пол                5000 non-null   int64  
 1   Возраст            5000 non-null   float64
 2   Зарплата           5000 non-null   float64
 3   Члены семьи        5000 non-null   int64  
 4   Страховые выплаты  5000 non-null   int64  
dtypes: float64(2), int64(3)
memory usage: 195.4 KB


None

,Пол,Возраст,Зарплата,Члены семьи,Страховые выплаты
count,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000
mean,0.499000,30.952800,39916.360000,1.194200,0.148000
std,0.500049,8.440807,9900.083569,1.091387,0.463183
min,0.000000,18.000000,5300.000000,0.000000,0.000000
25%,0.000000,24.000000,33300.000000,0.000000,0.000000
50%,0.000000,30.000000,40200.000000,1.000000,0.000000
75%,1.000000,37.000000,46600.000000,2.000000,0.000000
max,1.000000,65.000000,79000.000000,6.000000,5.000000


## Умножение матриц

Обозначения:

- $X$ — матрица признаков (нулевой столбец состоит из единиц)

- $y$ — вектор целевого признака

- $P$ — матрица, на которую умножаются признаки

- $w$ — вектор весов (коэффициентов) линейной регрессии (нулевой элемент равен сдвигу)

Для того чтобы зашифровать личные данные клиентов необходимо преобразовать левую часть исходного уравнения линейной регрессии 

$$
Xw = y
$$

для этого умножим исходную матрицу признаков на кодирующую матрицу $P$ (квадратная матрица, размер которой равен количеству столбцов в $X$)

$$
(XP)w_p = y
$$

При этом изменится вектор целевого признака так, чтобы сохранялось равенство 

$$
(XP)w_p = Xw = y
$$

Декодирование данных осуществляется умножение на обратную кодирующую матрицу $P^{-1}$

$$
X = (XP) P^{-1} = XE = X
$$

По этой причине кодирующая матрица должна быть обратима, в противном случае не удастся восстановить исходные данные.

Задача обучения линейной регрессии формулирует следующим образом:

$$
w = \underset{w}{\arg\min} MSE(Xw, y)
$$

Для нахождения метрики $R^2$ необходимо найти минимум функции $MSE(Xw, y)$ до и после шифрования данных, но поскольку $Xw$ до шифрования равен $(XP)w_p$ после шифрования, а вектор целевого признака остается неизменным, то метрики до и после шифрования будут равны, но при этом изменится вектор весов линейной регрессии.

Покажем как изменится вектор весов линейной регрессии в результате шифрования данных. 

Вектор коэффициентов в формуле обучения исходного уравнения имеет следующий вид:

$$
w = (X^T X)^{-1} X^T y
$$

После умножения левой части исходного уравнения линейной регрессии на кодирующую матрицу, вектор коэффициентов примет следующий вид:

$$
w_p = \Bigl((XP)^T (XP)\Bigr)^{-1} (XP)^T y 
$$

Преобразуем данное выражение:

$$
w_p = \Bigl(P^T (X^T XP)\Bigr)^{-1} (XP)^T y = \Bigl((X^T X) P\Bigr)^{-1} (P^T)^{-1} P^T X^T y
$$

$$
= P^{-1} (X^T X)^{-1} E X^T y = P^{-1} w
$$

Можно видеть, что

$$
w = P w_p
$$

Для того чтобы из коэффициентов линейной регрессии, полученной на зашифрованных данных, получить коэффиценты для исходных данных необходимо умножить их на кодирующую матрицу.

## Алгоритм преобразования

Создадим класс для шифрования и дешифровки данных.

In [4]:
class Encryption:
    def encrypt(self, X, random_seed=None):
        try:
            np.random.seed(random_seed)
            self.P = np.random.normal(size=(X.shape[1], X.shape[1]))
            self.P_inv = np.linalg.inv(self.P)
            self.X_encr = X @ self.P
        except np.linalg.LinAlgError:
            print('Кодирующая матрица является вырожденной')
    def decrypt(self, X):
        self.X_decr = self.X_encr @ self.P_inv

Создадим необходимые нам матрицы.

In [5]:
X = df.drop(columns=['Страховые выплаты']).values.astype('int')
# добавим нулевой столбец, состоящий из единиц
X = np.insert(X, 0, 1, axis=1)

y = df['Страховые выплаты'].values

Зашифруем матрицу признаков и вектор целевого признака, после этого расшифруем их.

In [6]:
coder = Encryption()
coder.encrypt(X, SEED)

X_encr = coder.X_encr

print('Размер матрицы признаков до и после шифрования:', 
      X.shape, X_encr.shape)

coder.decrypt(X)

X_decr = coder.X_decr

print('Доля совпадений после дешифровки матрицы признаков:', 
      (X == X_decr.round(0).astype('int')).mean())

Размер матрицы признаков до и после шифрования: (5000, 5) (5000, 5)
Доля совпадений после дешифровки матрицы признаков: 1.0


Можно видеть, что с помощью умножения матриц удается зашифровать и расшифровать имеющиееся данные. Однако при дешифровке приходится округлять данные до целых значений, т.к. при последовательном умножении матрицы признаков на $P$ и $P^{-1}$ данные в точности не совпадают. По-видимому, это связано с приближенными вычислениями матричного умножения и обратной матрицы.

## Проверка алгоритма

Определим коэффициенты линейной регрессии и величину метрики $R^2$ на исходных данных.

In [7]:
# fit_intercept=False, т.к. смещение является нулевым столбцом в матрице признаков
lr_0 = LinearRegression(fit_intercept=False)

lr_0.fit(X, y)
y_pred = lr_0.predict(X)

print('Величина R2-score на исходных данных:', r2_score(y, y_pred))
print('')
print('Величины коэффициентов линейной регрессии на исходных данных')
pd.DataFrame(lr_0.coef_.round(3), index=['Смещение'] + df.columns[:-1].to_list(), columns=['Значение'])

Величина R2-score на исходных данных: 0.42494550308169177

Величины коэффициентов линейной регрессии на исходных данных


,Значение
Смещение,-0.938
Пол,0.008
Возраст,0.036
Зарплата,-0.000
Члены семьи,-0.014


Определим коэффициенты линейной регрессии и величину метрики  $R^2$  на зашифрованных данных.

In [8]:
lr_1 = LinearRegression(fit_intercept=False)

lr_1.fit(X_encr, y)
y_encr_pred = lr_1.predict(X_encr)

print('Величина R2-score на зашифрованных данных:', r2_score(y, y_encr_pred))
print('')
print('Величины коэффициентов линейной регрессии на зашифрованных данных')
pd.DataFrame(data={'Значение': (lr_1.coef_).round(3), 
                   'После дешифровки': (coder.P @ lr_1.coef_).round(3)}, 
             index=['Смещение'] + df.columns[:-1].to_list())

Величина R2-score на зашифрованных данных: 0.42494550308309453

Величины коэффициентов линейной регрессии на зашифрованных данных


,Значение,После дешифровки
Смещение,-0.877,-0.938
Пол,-4.115,0.008
Возраст,-2.217,0.036
Зарплата,1.557,-0.000
Члены семьи,-0.578,-0.014


Можно видеть, что величины R2-score на исходных и зашифрованных данных совпадают, а величины коэффициентов, как и ожидалось отличаются, но их можно дешифровать с помощью кодирующей матрицы. 

Сравним величины предсказанных значений на исходных данных и на данных после шифрования.

In [9]:
print('Доля совпадающих предсказанных значений до и после шифрования:',
(y_pred.round(0).astype('int') == 
(y_encr_pred).round(0).astype('int')).mean())

Доля совпадающих предсказанных значений до и после шифрования: 1.0


## Выводы
- с  помощью матричного умножения удается зашифровать данные для последующего использования в модели линейной регрессии, а также расшифровать их
- метрики моделей $R^2$ на исходных и зашифрованных данных практически совпадают. Это говрит о том, что защита личных данных не ухудшает качества модели
- расхожения в величинах найденных метрик и данных после операции шифрование-дешифровка можно связать с применяемыми приближениями в матричном умножении и в нахождении обратной матрицы